---
Finding Bottlenecks
---

Now that we have learned some techniques with no to minor code modifications, we might want to tweak our actual code itself since there are chances we can do better. The way to determine what part of our code needs attention is called profiling. Profilers come in a variety of shapes and functionalities but what we'll use are two deterministic/event-based profilers as opposed to statistical profilers. This means that instead of relying on partial data by sampling at regular intervals, hit count will be exact and can be reproduced. The downside to using an event-based profiler is that is can slow down your code, sometimes significantly.

Here are some things to keep in mind before optimizing:

* Make sure your code works/is correct beforehand.
* Make sure you focus on the right code by profiling.
* Make and keep performance measurements along the way.
* Make the smallest changes possible at a time.
* Keep track of your code changes using Git.
* Always make sure your unit tests/your output is still valid.

We'll start with a simple example that can also be found in the profile.py file:

In [ ]:
import random

random.seed(0)

def gen_data(n):
    numbers = []
    for i in range(n):
        numbers.append(random.random())
    return numbers

def sum_nexts(numbers):
    sums = []
    for i in range(len(numbers)):
        for j in range(i+1, len(numbers)):
            if len(sums) < i+1:
                sums.append(0.)
            sums[i] = sums[i] + numbers[j]
    return sums

def profile_sum(n):
    numbers = gen_data(n)
    sums = sum_nexts(numbers)
    return sums

This small program takes a number n as input, generates n random elements in a list called numbers, and compute a list of n elements called sums, such as:

$$ {\Large \mathrm{sum}_i = \sum_{a=i+1}^{n-1} \mathrm{number}_a } $$

The first thing we want to do is make note of the time it takes to run the program. We'll start small, with a run of 10000 generated elements.

In [ ]:
%timeit profile_sum(10000)

We're now ready to make our first profiling measurement to find our (possible) bottlenecks. For that, we'll use the recommended Python profiler: [cProfile](https://docs.python.org/2/library/profile.html#module-cProfile)

In [ ]:
?%prun

In [ ]:
%prun profile_sum(10000)

You must remember that you can't compare run timings from profiled and non-profiled code. Profiling incurs an overhead and will usually be a lot slower (about 50% here). The default output is not useful as entries are in no particular order. What we want most of the time is function calls sorted by their reverse cumulative time. This can be accomplished by the following:

In [ ]:
%prun -s cumulative profile_sum(10000)

It's then possible to infer that most of the computation time, as expected, is taking place in the sum_nexts function. Beware that it might not always be so predictable and that you must profile before starting optimizing your code.

To save the output in a variable we can use -q to suppress screen output and -r to return the stats output:

In [ ]:
stats = %prun -q -r profile_sum(10000)

This is almost always what you want when profiling a real software as there will be a lot of functions to display.

The problem when using cProfile is that it's view is very low-level. We know where to start looking, but we don't know the details of the function execution. To dig further, we need to use a line-based profiler called line_profiler, provided by the %lprun IPython magic.

In [ ]:
def sum_nexts(numbers):
    sums = []
    for i in range(len(numbers)):
        for j in range(i+1, len(numbers)):
            if len(sums) < i+1:
                sums.append(0.)
            sums[i] = sums[i] + numbers[j]
    return sums

We can load, get help on, start the profiler and run our program using the following:

In [ ]:
%load_ext line_profiler

In [ ]:
?%lprun

In [ ]:
%lprun -f sum_nexts profile_sum(10000)

This makes it clear that a lot of the time is consumed by the (innocuously looking) if len(sums) < i+1 statement. It is not slow per say, but called 50 million times, it becomes significant. A simple change we can make is to initialize our result list before with zeros. This change would look like this:

In [ ]:
def sum_nexts(numbers):
    sums = [0.]*len(numbers)
    for i in range(len(numbers)):
        for j in range(i+1, len(numbers)):
            sums[i] = sums[i] + numbers[j]
    return sums

We want to measure the impact of our change, by comparing to the original run time:

In [ ]:
%timeit profile_sum(10000)

So we have a 50% gain just doing this small modification. Let's see if we can do better. We run the line-level profiler again:

In [ ]:
%lprun -f sum_nexts profile_sum(10000)

That inner loop (j variable) sure seems to be a problem. Instead of iterating from i+1 to len(number)-1, we could use Python splicing to see if we get better performance.

In [ ]:
def sum_nexts(numbers):
    sums = [0.]*len(numbers)
    for i in range(len(numbers)):
        sums[i] = sum(numbers[i+1:])
    return sums

We can test our hypothesis:

In [ ]:
%timeit profile_sum(10000)

That's great! A 20x improvement since the beginning of the process. Let's see what our line-based profiler has to say:

In [ ]:
%lprun -f sum_nexts profile_sum(10000)

This is what we want to see: almost all run time is the actual computation (sum), which is great. But still, since we are working with lists, why not use Numpy arrays and see what we can achieve? Here is a modified version using Numpy arrays instead is lists:

In [ ]:
import numpy
import time

numpy.random.seed(int(time.time()))

def gen_data(n):
    return numpy.random.random(n)

def sum_nexts(numbers):
    sums = numpy.zeros(len(numbers))
    for i in range(len(numbers)):
        sums[i] = sum(numbers[i+1:])
    return sums

But running it yields an unexpected result. We are far worse than we were before those modifications:

In [ ]:
%timeit profile_sum(10000)

Why? Let's ask our profiler:

In [ ]:
%lprun -f sum_nexts profile_sum(10000)

This is again expected, we spend all of our time doing the sum. But why is it slower? The answer is that we are still using the Python built-in sum function and not it's Numpy counterpart. That means the code goes back and forth between Python and C (Numpy). We can make the code execute longer in the Numpy library by using its own sum function:

In [ ]:
def sum_nexts(numbers):
    sums = numpy.zeros(len(numbers))
    for i in range(len(numbers)):
        sums[i] = numbers[i+1:].sum()
    return sums

In [ ]:
%timeit profile_sum(10000)

So in the end, we could get about 65 times faster in a really short amount of time.

> ## More Data Processing {.challenge}
>
> Try optimizing the below code by first using a profiler and finding hotspots.
>
> __Tip__: Before running the script, generate a random sample.
>

In [ ]:
import random
random.seed(1234)
with open("inputs.dat", "w") as fp:
    for _ in range(5000):
        for _ in range(5000):
            fp.write("{0},".format(random.random()))
        fp.write("\n")


In [ ]:
# %load cq-formation-advanced-python/exercices/process_data.py
from __future__ import division

def read_data():
    data = []
    fp = open("inputs.dat", "r")

    line = 1
    while line:
        line = fp.readline()
        if line:
            row = []
            for elem in line.split(','):
                elem = elem.strip()
                if elem:
                    row.append(float(elem))
            data.append(row)
        
    fp.close()
    return data

def process_A(data):
    """
    Return a new matrix of the same shape as data, with each original
    element squared by it's transposition equivalent.

    result[i][j] = data[i][j] ** data[j][i]
    """
    result = []
    for i in range(len(data)):
        row = []
        for j in range(len(data[i])):
            row.append(data[i][j] ** data[j][i])
        result.append(row)
    return result

def process_B(m1, m2):
    """
    Return the sum of the difference between each corresponding
    elements of two square matrices.

    diff = (m2[0][0] - m1[0][0]) + (m2[0][1] - m1[0][1]) + ...
    """

    diff = 0.
    for i in range(len(m1)):
        for j in range(len(m1[i])):
            diff += m2[i][j] - m1[i][j]
    return diff

def main():
    data = read_data()
    result_1 = process_A(data)
    print("Difference is: %s"%process_B(data, result_1))

In [ ]:
main()

> A possible solution can be found in the solutions/process_data.py file.